# Random walks and tabs from computed Markov chains

In [1]:
import networkx as nx
import numpy as np
import guitarpro as gp
import copy

from custom_functions import *

In [3]:
song = gp.parse("Ginpatsu_no_shounen_Killua.gp5", encoding='UTF-8')

In [4]:
def render_tab(walk, tuning=['e','A','D','G','B','E'], measure = ['|','|','|','|','|','|'], separator = ['--','--','--','--','--','--']):
    '''
    From a random walk, write the corresponding tab.
    '''
    
    def transpose(M):
        return [[M[j][i] for j in range(len(M))] for i in range(len(M[0]))]

    n = len(walk)
    
    T = [tuning, measure, measure, separator]
    
    # Separate the walk in beats
    for beat in walk:
        
        # Initialize the vertical line to add
        vertline = copy.copy(separator)

        # Count the notes with single digit frets
        single_digit = 0
        
        # Add the corresponding notes to the vertical line
        for subnote in beat.split('-'):
            string = letter_to_string(subnote[-1])
            fret = str(subnote[:-1])
            
            # Convert single digits to two digits
            if len(fret) == 1:
                fret = '-' + fret
                single_digit += 1
            
            vertline[string] = fret
            
        # Remove the extra dash if all of the notes in the 
        # vertical line have single digit frets
        if single_digit == len(beat.split('-')):
            vertline = [v[1] for v in vertline]  
        
        T.append(vertline)
        T.append(separator)
    
    T = transpose(T)    
    
    tab = ''
    
    for line in T:
        tab += ''.join(line) + '\n'
    
    return tab
    

## Memory-less

In [5]:
G = construct_chain(song)

In [6]:
def random_walk(G, start, length):
    '''
    Perform a random walk in G
    of length 'length' starting at 'start'.
    '''
    
    A = nx.to_numpy_array(G)
    nodelist = list(G.nodes())
    
    if start not in nodelist:
        raise Exception('Choose a valid start')
    
    i = 0
    walk = [start]
    
    while i < length:
        prob = A[nodelist.index(start)]
        start = np.random.choice(nodelist, 1, p=prob)[0]
        
        walk.append(start)
        
        i+=1
        
    return walk        

In [7]:
walk = random_walk(G, '0e', 10)
walk

['0e',
 '3e',
 '5e-6B',
 '8e-6B',
 '10e-10B',
 '8e',
 '7e',
 '5e-5B',
 '15e-17B-17G',
 '12e-15B-14G',
 '16e-17B']

In [8]:
tab = render_tab(walk)
print(tab)

e||--0--3--5--8--10--8--7--5--15--12--16--
A||--------6--6--10--------5--17--15--17--
D||---------------------------17--14------
G||---------------------------------------
B||---------------------------------------
E||---------------------------------------



## With memory (of any order)

In [11]:
LdG = line_datagraph(song)

Traccia 1
Measure key: CMajor
Measure time signature: 3/4


In [33]:
def random_walk_withmemory(LdG, start, length):
    '''
    Perform a random walk with memory in G
    of length 'length' starting at 'start'.
    Note: 'start' should be provided as n
    consecutive notes joined by 'x'. 
    '''
    
    A = nx.to_numpy_array(LdG)
    nodelist = list(LdG.nodes())
    
    if start not in nodelist:
        raise Exception('Choose a valid start')
    
    i = 0
    walk = [*start.split('x')]
    
    while i < length:
        prob = A[nodelist.index(start)]
        start = np.random.choice(nodelist, 1, p=prob)[0]
        
        next_node = start.split('x')[-1]
        
        walk.append(next_node)
        
        i += 1
        
    return walk

In [32]:
walk = random_walk_withmemory(LdG, '0ex3e', 10)
tab = render_tab(walk)
print(tab)

e||--0--3--5--3--0--------5--8--10--8--7--
A||--------6--5--3--------6--6--10--------
D||-----------------5--7------------------
G||--------------------9------------------
B||---------------------------------------
E||---------------------------------------

